# Turn Team summary (in txt file) into the Strategy Reponse.

First run the pdf_txt_pipeline to extract information from the match reports.

1. **Setup OpenAI Client**:
   - The code initializes the OpenAI API client by providing an API key.

2. **Read Team Profiles**:
   - It looks for `.txt` files in a specified folder (`./school_1_match`).
   - Each file represents a team's profile for the 2024 season.
   - It reads and combines the content of all these files into a single formatted string.

3. **Create GPT Query**:
   - The combined profiles are included in a query message for GPT, formatted to ask specific questions:
     - Summarize each team's strengths and weaknesses.
     - Suggest strategies for Northwestern to play against each team.
     - Recommend three statistics to focus on based on the strategy.

4. **Send Query to GPT**:
   - The formatted query is sent to GPT-4 for processing.
   - It uses `max_tokens=5000` to allow for detailed responses.

5. **Display GPT Response**:
   - The code retrieves GPT’s response and prints it to the console.
   - It optionally stores the response for further use.

In [ ]:
import os
from openai import OpenAI

# OpenAI API setup
client = OpenAI(api_key="your_api_key")

# Define the school_txt_profiles directory, could be anything
profiles_folder = "./school_1_match"

# Function to concatenate all team profiles
def concatenate_team_profiles(folder):
    all_profiles = ""
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".txt"):
            team_name = filename.replace("-2024.txt", "").replace("-", " ").title()
            with open(os.path.join(folder, filename), "r") as file:
                content = file.read()
                all_profiles += f"### {team_name}\n{content}\n\n"
    return all_profiles

# Concatenate profiles into a single string
profiles_content = concatenate_team_profiles(profiles_folder)

# Prepare the query for GPT
query_message = [
    {
        "role": "user",
        "content": f"""
You are a soccer analysis expert. Below are team profiles for the 2024 season.

{profiles_content}

For each team:
1. Summarize the team's strengths in 2-3 detailed bullet points with some examples
2. Summarize the team's weaknesses in 2-3 detailed bullet points with some examples.
3. Provide a strategy for how Northwestern can play against each team by HIGHLIGHTING how our STRENGTH relate to their weaknesses in 2-3 bullet points.
4. Out of these columns, give me 3 recommended stats to look at BASED ON the strategy you recommended:
    'pass_success_rate', 'possession', 'tot_duels_win_rate',
    'sobotr', 'cornwsr', 'fkwsr', 'pen_conversion_rate', 'acc_cross_rate',
    'succ_for_passes_rate', 'succ_back_passes_rate', 'succ_lat_passes_rate',
    'succ_long_passes_rate', 'succ_fin_third_passes_rate', 'succ_prog_passes_rate',
    'succ_smart_passes_rate', 'succ_throw_ins_rate', 'mean_shot_dist',
    'mean_pass_len', 'ppda', 'passes_per_minute', 'goal_conversion_rate',
    'touches_in_box_per_minute', 'recoveries_per_minute', 'shots_on_target_rate'. This will help the coach decide what stats to look at.

Provide your response in the format:
- SCHOOL NAME: **Team Name:**
  - Strengths:
    - Bullet 1
    - Bullet 2
    - Bullet 3
  - Weaknesses:
    - Bullet 1
    - Bullet 2
    - Bullet 3
  - Northwestern's Strategy:
    - Bullet 1
    - Bullet 2
    - Bullet 3
  - Recommended Columns: [Column 1, Column 2, Column 3]

"""
    }
]

# Query the LLM
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=query_message,
    max_tokens=5000,
)

# Print the response
print("Response from the LLM:")
print(response.choices[0].message.content)
overall_strategy = response.choices[0].message.content

Response from the LLM:
- SCHOOL NAME: **Indiana Hoosiers:**
  - Strengths:
    - **Central Control:** Indiana's focus on passing through the central midfield allows them to maintain possession and control the game's tempo, making it difficult for opponents to win the ball back. This strategy can create gaps in the opposing defense.
    - **Short Combinations:** Their tendency for quick, short passes enables them to link play effectively, disrupting defensive organization and creating goal-scoring chances during quick exchanges.
    - **High Duel Win Ratio in Attack:** Indiana shows strength in duels within their attacking third, especially on the right side, where they have a favorable win/loss ratio, aiding in possession retention and penetration.
  - Weaknesses:
    - **Limited Width:** By concentrating their play in central areas, they may struggle to stretch defenses, making them vulnerable to teams that can defend compactly.
    - **Predictability:** Their reliance on central pass

In [ ]:
##

## Save Strategy response into txt file

In [ ]:
# Define the output directory for individual files
output_directory = "team_analysis_schools_1_updated"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Split the text by each school's section
teams = overall_strategy.split("- SCHOOL NAME: ")[1:]  # Split by the SCHOOL NAME delimiter

for team_data in teams:
    # Extract team name and its content
    lines = team_data.strip().splitlines()
    team_name = lines[0].strip()  # The first line is the team name
    team_analysis = "\n".join(lines[1:])  # The rest of the lines contain the analysis

    # Write each team's analysis into a separate file
    file_path = os.path.join(output_directory, f"{team_name}.txt")
    with open(file_path, "w") as file:
        file.write(f"- **SCHOOL NAME: {team_name}**\n{team_analysis}")

print(f"Team analyses have been saved to the '{output_directory}' directory.")

Team analyses have been saved to the 'team_analysis_schools_1_updated' directory.


In [ ]:
import os
# Define the output directory for individual files
output_directory = "school_all_match"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
import os

# Folder containing the files
folder_path = "school_profiles_5"

# Define mapping for renaming
rename_mapping = {
    "__Indiana___.txt": "Indiana_Hoosiers__.txt",
    "__Maryland___.txt": "Maryland_Terrapins__.txt",
    "__Michigan State___.txt": "Michigan_State_Spartans__.txt",
    "__Michigan___.txt": "Michigan_Wolverines__.txt",
    "__Northwestern___.txt": "Northwestern_Wildcats__.txt",
    "__Ohio State___.txt": "Ohio_State_Buckeyes__.txt",
    "__Penn State___.txt": "Penn_State_Nittany_Lions__.txt",
    "__Rutgers___.txt": "Rutgers_Scarlet_Knights__.txt",
    "__UCLA___.txt": "UCLA_Bruins__.txt",
    "__Washington___.txt": "Washington_Huskies__.txt",
    "__Wisconsin___.txt": "Wisconsin_Badgers__.txt"
}

# Iterate through files and rename based on the mapping
for old_name, new_name in rename_mapping.items():
    old_file_path = os.path.join(folder_path, old_name)
    new_file_path = os.path.join(folder_path, new_name)

    # Rename the file if it exists
    if os.path.exists(old_file_path):
        os.rename(old_file_path, new_file_path)
        print(f"Renamed: {old_file_path} -> {new_file_path}")
    else:
        print(f"File not found: {old_file_path}")
